
<h2> Survival Analysis (Kaplan Meier) Network Study Using Strategus</h2>

<h4> Step 1 - Import the necessary libraries </h4>


In [ ]:
%%jupyter
# import libraries
library(rD2E)
library(Strategus)
library(dplyr)

<h4> Step 2 - Cohort definition loading </h4>

In [ ]:
%%jupyter
cohorts_set <- c(13)
cohortDefinitionSet <- rD2E::get_cohort_definition_set(cohort_set)

<h4> Step 3 - Define the network study components </h4>

In [ ]:
%%jupyter
# Target-Comparator pairs
cmTcList <- data.frame(
  targetCohortId = 1001,
  targetCohortName = "Immunotherapy + Surgery",
  comparatorCohortId = 1002,
  comparatorCohortName = "Surgery only"
)

# Outcome cohort
outcomeCohortId <- 3001

# Set time-at-risk
timeAtRisks <- tibble(
  label = c("KM Analysis"),
  riskWindowStart = c(1),
  startAnchor = c("cohort start"),
  riskWindowEnd = c(0),
  endAnchor = c("cohort end")
)

# Define the outcome
outcomeList <- lapply(seq_len(1), function(i) {
  CohortMethod::createOutcome(
    outcomeId = outcomeCohortId,
    outcomeOfInterest = TRUE
  )
})

# Define the T-C-O structure
targetComparatorOutcomesList <- list(
  CohortMethod::createTargetComparatorOutcomes(
    targetId = cmTcList$targetCohortId,
    comparatorId = cmTcList$comparatorCohortId,
    outcomes = outcomeList
  )
)

<p> Create settings for modules involved in the study </p>

In [ ]:
%%Jupyter
# Setup cohort method module
cmModuleSettingsCreator <- CohortMethodModule$new()

cmAnalysisList <- list(
  CohortMethod::createCmAnalysis(
    analysisId = 1,
    description = "Kaplan-Meier survival analysis",
    getDbCohortMethodDataArgs = CohortMethod::createGetDbCohortMethodDataArgs(
      studyStartDate = studyStartDate,
      studyEndDate = studyEndDate
    ),
    createStudyPopArgs = CohortMethod::createCreateStudyPopulationArgs(
      firstExposureOnly = TRUE,
      removeDuplicateSubjects = "keep first",
      removeSubjectsWithPriorOutcome = FALSE,
      priorOutcomeLookback = 0,
      requireTimeAtRisk = FALSE,
      riskWindowStart = timeAtRisks$riskWindowStart,
      startAnchor = timeAtRisks$startAnchor,
      riskWindowEnd = timeAtRisks$riskWindowEnd,
      endAnchor = timeAtRisks$endAnchor
    )
  )
)

cohortMethodModuleSpecifications <- cmModuleSettingsCreator$createModuleSpecifications(
  cmAnalysisList = cmAnalysisList,
  targetComparatorOutcomesList = targetComparatorOutcomesList
)

# Cohort Generator Module
cgModuleSettingsCreator <- CohortGeneratorModule$new()
cohortDefinitionShared <- cgModuleSettingsCreator$createCohortSharedResourceSpecifications(cohortDefinitionSet)
cohortGeneratorModuleSpecifications <- cgModuleSettingsCreator$createModuleSpecifications()

<p> Create the analysis specification object - including all the modules and configurations created above </p>

In [ ]:
%%jupyter
# Final Analysis Spec
analysisSpecifications <- createEmptyAnalysisSpecificiations() |>
  addSharedResources(cohortDefinitionShared) |>
  addModuleSpecifications(cohortGeneratorModuleSpecifications) |>
  addModuleSpecifications(cohortMethodModuleSpecifications)

<h4> Step 4 - Execute the Strategus study </h4>

In [ ]:
%%jupyter
study_name <- "survival_analysis_study"  # Unique study name
options <- create_options(upload_results=TRUE, study_id = study_name) # set a study_id with a unique id
rD2E::run_strategus_flow(analysisSpecification = analysisSpecifications, options = options)